In [ ]:
import pandas as pd
import numpy as np
# pip install patsy
from patsy import dmatrices, dmatrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(font_scale=1.5)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
y, X = dmatrices('Survived ~ Age + C(Pclass) + C(Sex)', train, return_type='dataframe')
# flatten y so we can incorporate in sklearn functions
y = np.ravel(y)

In [ ]:
X.head()

In [ ]:
# standardize the features since regularization requires all features to be on same scale
scaler = StandardScaler(copy=True)
# we have created a standardization based on the training data
X_clean = scaler.fit(X).transform(X)

In [ ]:
pd.DataFrame(X_clean, columns=X.columns).head()

In [ ]:
# build our initial model
lr = LogisticRegression('l1')
lr.fit(X_clean, y)
print (lr.coef_)

In [ ]:
# a sparse girl
girl = [1., 0., 0., 0., 21.]
girl = scaler.fit(X).transform(girl)
lr.predict_proba(girl.reshape(1,-1))

In [ ]:
print (girl)

In [ ]:
lr.intercept_

In [ ]:
lr.coef_[0]

In [ ]:
#this is it
logodds = lr.intercept_ +(-0.50102192* -0.55059023)+(-1.29833678*-1.0121585)+(-1.17900718*-1.33473316)+\
(-0.56806052* -0.59550826)
logodds
# convert log-odds to odds
odds = np.exp(logodds)
odds
prob = odds/(1 + odds)
prob

In [ ]:
def sigmoid(x):
    return 1/(1+np.e**(-x))

In [ ]:
sigmoid( np.dot(girl, lr.coef_.T) + lr.intercept_ )

In [ ]:
# compute predicted log-odds for al=2 using the equation
logodds = logreg.intercept_ + logreg.coef_[0] * 2
logodds

In [ ]:
# convert log-odds to odds
odds = np.exp(logodds)
odds

In [ ]:
# convert odds to probability
prob = odds/(1 + odds)
prob

In [ ]:
# compute predicted probability for al=2 using the predict_proba method
lr.predict_proba(girl.reshape(1,-1))[:, 1]